<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/mrc_LSTM_baseline0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Libraries and Read Data

In [35]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
project_path = "/content/drive/My Drive/AIML-MRC-Capstone/datasets/Squad2.0/TrainingDataset/"

squad_df = pd.read_csv(project_path+'squad_data_final.csv')
squad_df.drop('Unnamed: 0',axis=1,inplace=True)
squad_df.tail(5)

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible,clean_context,clean_question,clean_answer
130301,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,5a7e070b70df9f001a875439,NaN,NaN,485.0,matter,True,term matter used throughout physics bewilderin...,physics has broadly agreed on the definition o...,IMPOSSIBLE
130302,Matter,"The term ""matter"" is used throughout physics i...",Who coined the term partonic matter?,5a7e070b70df9f001a87543a,NaN,NaN,327.0,Alfvén,True,term matter used throughout physics bewilderin...,who coined the term partonic matter,IMPOSSIBLE
130303,Matter,"The term ""matter"" is used throughout physics i...",What is another name for anti-matter?,5a7e070b70df9f001a87543b,NaN,NaN,350.0,Gk. common matter,True,term matter used throughout physics bewilderin...,what is another name for antimatter,IMPOSSIBLE
130304,Matter,"The term ""matter"" is used throughout physics i...",Matter usually does not need to be used in con...,5a7e070b70df9f001a87543c,NaN,NaN,529.0,a specifying modifier,True,term matter used throughout physics bewilderin...,matter usually does not need to be used in con...,IMPOSSIBLE
130305,Matter,"The term ""matter"" is used throughout physics i...",What field of study has a variety of unusual c...,5a7e070b70df9f001a87543d,NaN,NaN,37.0,physics,True,term matter used throughout physics bewilderin...,what field of study has a variety of unusual c...,IMPOSSIBLE


# Lets use 2 LSTM network for context and question and use a concat layer to merge. y is the answer 

## Step 1 Tokenize clean_context

In [0]:
context_tokenize = tf.keras.preprocessing.text.Tokenizer()
context_tokenize.fit_on_texts(squad_df['clean_context']) #Fit it on clean_context

In [37]:
#check vocab size of clean_context
len(context_tokenize.word_index)

93529

In [38]:
print(context_tokenize.word_index)

{'also': 1, 'one': 2, 'first': 3, 'new': 4, 'many': 5, 'city': 6, 'states': 7, 'used': 8, 'two': 9, 'may': 10, 'time': 11, 'united': 12, 'state': 13, 'world': 14, 'century': 15, 'would': 16, 'war': 17, 'known': 18, 'years': 19, 'however': 20, 'including': 21, 'use': 22, 'system': 23, 'government': 24, 'early': 25, 'people': 26, 'became': 27, 'since': 28, 'later': 29, 'made': 30, 'well': 31, 'number': 32, 'part': 33, 'often': 34, 'called': 35, 'million': 36, 'national': 37, 'north': 38, 'three': 39, '1': 40, 'power': 41, 'population': 42, 'year': 43, 'university': 44, 'several': 45, 'major': 46, 'american': 47, 'although': 48, 'music': 49, 'south': 50, 'area': 51, 'period': 52, 'british': 53, 'u': 54, 'include': 55, 'large': 56, 'law': 57, 'high': 58, 'within': 59, 'public': 60, 'form': 61, 'school': 62, 'language': 63, 'around': 64, 'began': 65, '2': 66, 'west': 67, 'church': 68, 'among': 69, 'second': 70, 'could': 71, 'led': 72, 'french': 73, 'political': 74, 'found': 75, 'great': 76,

In [0]:
#convert the context text to indexes
context_sequence= context_tokenize.texts_to_sequences(squad_df['clean_context'])


In [40]:
squad_df['clean_context'][3000]

'office buildings shanghais financial district including jin mao tower hong kong new world tower evacuated receptionist tibet hotel chengdu said things calm hotel evacuated guests meanwhile workers ford plant sichuan evacuated 10 minutes chengdu shuangliu international airport shut control tower regional radar control evacuated one silkair flight diverted landed kunming result cathay pacific delayed legs quadruple daily hong kong london route due disruption air traffic services chengdu shuangliu airport reopened later evening may 12 offering limited service airport began used staging area relief operations'

In [0]:
context_sequence[3000]

# Step 2 Find out max sequence length of clean_context

In [40]:
# max length of context
max_context_seq_length= max(len(txt) for txt in context_sequence)
print( max_context_seq_length)

# vocab size of context
context_vocab_size=len(context_tokenize.word_index)
print(context_vocab_size)

426
93529


# Step 3 Padding the sequence for clean_context

In [0]:
context_input_data= tf.keras.preprocessing.sequence.pad_sequences(context_sequence, maxlen=max_context_seq_length, padding='pre')

# Step 4 Build the LSTM Model for 'Context' and get the c and h values

In [0]:
embedding_size = 50
rnn_units=256


In [0]:
# input layer
context_input=tf.keras.layers.Input(shape=(32,))

# Build Embedding layer
context_embedding=tf.keras.layers.Embedding(context_vocab_size+1, embedding_size)

# Get Embedding Layer output
context_embedding_output=context_embedding(context_input)

#LSTM layer and its output
lstm1= tf.keras.layers.LSTM(rnn_units,return_state=True)(context_embedding_output)

#build a list to feed for concatenation
#context_states= [state_h1, state_c1]

In [10]:
context_embedding_output.shape

TensorShape([None, 32, 50])

# Step 5 Tokenize the Questions

In [0]:
questions_tokenize= tf.keras.preprocessing.text.Tokenizer()
questions_tokenize.fit_on_texts(squad_df['clean_question'])

In [45]:
#check vocab size of questions
len(questions_tokenize.word_index)

47289

In [46]:
print(questions_tokenize.word_index)

{'the': 1, 'what': 2, 'of': 3, 'in': 4, 'to': 5, 'was': 6, 'is': 7, 'did': 8, 'a': 9, 'who': 10, 'how': 11, 'for': 12, 'and': 13, 'when': 14, 'which': 15, 'are': 16, 'many': 17, 'on': 18, 'does': 19, 'were': 20, 'with': 21, 'that': 22, 'by': 23, 'from': 24, 'where': 25, 'as': 26, 'year': 27, 'do': 28, 'have': 29, 'be': 30, 'first': 31, 'name': 32, 'an': 33, 'not': 34, 'at': 35, 'used': 36, 'has': 37, 'type': 38, 'can': 39, 'most': 40, 'during': 41, 'new': 42, 'people': 43, 'other': 44, 'city': 45, 'their': 46, 'after': 47, 'one': 48, 'it': 49, 'why': 50, 'use': 51, 'its': 52, 'two': 53, 'much': 54, 'called': 55, 'his': 56, 'about': 57, 'century': 58, 'war': 59, 'been': 60, 'or': 61, 'time': 62, 'had': 63, 'country': 64, 'between': 65, 'group': 66, 'term': 67, 'system': 68, 'into': 69, 'state': 70, 'located': 71, 'part': 72, 'world': 73, 'made': 74, 'language': 75, 'known': 76, 'more': 77, 'kind': 78, 'there': 79, 'some': 80, 'than': 81, 'population': 82, 'percentage': 83, 'states': 84,

In [0]:
# convert the questions to indexes
questions_sequence= questions_tokenize.texts_to_sequences(squad_df['clean_question'])

In [48]:
squad_df['clean_question'][2000]

'who was chief financial officer of apple in july of 2009'

In [49]:
questions_sequence[2000]

[10, 6, 886, 620, 2250, 3, 762, 4, 1224, 3, 320]

# Step 6 Find out maximum sequence length of Questions

In [51]:
max_question_seq_length=max(len(txt) for txt in questions_sequence)
print(max_question_seq_length)

# vocab size of questions
questions_vocab_size=len(questions_tokenize.word_index)
print(questions_vocab_size)

40
47289


# Step 7 Padding the sequences for Questions

In [0]:
question_input_data=tf.keras.preprocessing.sequence.pad_sequences(questions_sequence, maxlen=max_question_seq_length,padding='pre')

# Step 8 Build the LSTM Model for 'Questions' and get the c and h values

In [0]:
#input layer
question_input=tf.keras.layers.Input(shape=(32,))

#Embedding layer
question_embedding=tf.keras.layers.Embedding(questions_vocab_size+1, embedding_size)

#Embedding layer output

question_embedding_output=question_embedding(question_input)

#LSTM2 layer 
lstm2= tf.keras.layers.LSTM(rnn_units,return_state=True)(question_embedding_output)

In [54]:
question_embedding_output.shape

TensorShape([None, 32, 50])

# Concat the two LSTM layers

In [0]:
concat=tf.keras.layers.concatenate([lstm1,lstm2])

# Create Decoder for Answer

# Add  Start and  End tokens to Answers

In [0]:
squad_df['answer_start_end']= '<start>' + squad_df['clean_answer'] + '<end>'


In [0]:
squad_df['answer_start_end']=squad_df['answer_start_end'].astype(str)


#Tokenize the Answers

In [0]:
answers_tokenize=tf.keras.preprocessing.text.Tokenizer()
answers_tokenize.fit_on_texts(squad_df['answer_start_end'])

In [58]:
#Vocab
len(answers_tokenize.word_index)

41475

In [25]:
print(answers_tokenize.word_index)

{'end': 1, 'start': 2, 'impossible': 3, 'the': 4, 'of': 5, 'and': 6, 'a': 7, 'to': 8, 'in': 9, 'or': 10, 'for': 11, 'million': 12, 'as': 13, 'by': 14, 'on': 15, 'is': 16, 'with': 17, 'from': 18, 'that': 19, 'an': 20, 'new': 21, 'century': 22, 'two': 23, 'their': 24, 'one': 25, 'was': 26, 'united': 27, '1': 28, 'at': 29, 'not': 30, 'be': 31, 'years': 32, 'more': 33, 'are': 34, 'three': 35, '2': 36, 'it': 37, 'states': 38, 'war': 39, '5': 40, 'over': 41, 'world': 42, 'than': 43, 'its': 44, '3': 45, 'his': 46, 'state': 47, 'four': 48, 'national': 49, 'john': 50, 'may': 51, 'city': 52, 'between': 53, 'all': 54, 'de': 55, 'were': 56, 'system': 57, 'other': 58, 'university': 59, 'british': 60, 'government': 61, 'billion': 62, 'they': 63, '4': 64, 'five': 65, '8': 66, 'first': 67, 'north': 68, 'bc': 69, '6': 70, 'have': 71, 'international': 72, 'no': 73, 'south': 74, 'general': 75, '7': 76, 'law': 77, 'power': 78, 'french': 79, '10': 80, 'east': 81, 'early': 82, 'high': 83, 'which': 84, '20':

In [0]:
answers_seq = answers_tokenize.texts_to_sequences(squad_df['answer_start_end']) #Convert sentences to numbers 

In [27]:
answers_seq[2000]

[2, 924, 7781, 1]

# Get maximum length and pad the sequences

In [60]:
max_answers_seq_length=max(len(txt) for txt in squad_df['answer_start_end'])
print(max_answers_seq_length)

answers_vocab_size=len(answers_tokenize.word_index)
print(answers_vocab_size)

248
41475


In [0]:
answers_input_data= tf.keras.preprocessing.sequence.pad_sequences(answers_seq,maxlen=max_answers_seq_length,padding='pre')

# Building Decoder Output

In [62]:
answers_input_data.shape

(130306, 248)

In [0]:
#Initialize array
answers_target_data = np.zeros((answers_input_data.shape[0], #number of sentences 130306
                                answers_input_data.shape[1])) #number of words in each sentence 248

#Shift Target output by one word
for i in range(answers_input_data.shape[0]):
    for j in range(1,answers_input_data.shape[1]):
        answers_target_data[i][j-1] = answers_input_data[i][j]

In [31]:
squad_df['answer_start_end'][0]

'<start>in the late 1990s<end>'

In [32]:
answers_input_data[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [33]:
answers_target_data[0]

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   

# Convert Answers to one-hot vector

In [0]:
#answers_target_data_one_hot= np.zeros((answers_input_data.shape[0], 
                                       answers_input_data.shape[1],len(answers_tokenize.word_index)+1))

In [0]:
answers_embedding_size = 50
rnn_units = 256

# Build Decoder

In [0]:
#input layer
answers_inputs=tf.keras.layers.Input(shape=(None,))

#Embedding
answers_embedding=tf.keras.layers.Embedding(answers_vocab_size+1, answers_embedding_size)
answers_embedding_output=answers_embedding(answers_inputs)

#lstm layer
answers_lstm= tf.keras.layers.LSTM(rnn_units,return_sequences=True, return_state=True)

#LSTM Output, State initialization from Encoder states(concat of question and answer)
#Output will be all hidden sequences, last 'h' state and last 'c' state

lstm3=answers_lstm(answers_embedding_output,initial_state=<concatlayer h and c)

#dense layer
lstm3_dense= tf.keras.layers.Dense(answers_vocab_size+1,activation='softmax',)

#answer output
answer_outputs=lstm3_dense(lstm3)

# Build Model using Encoder ( output of concat) and Decoder

# Train the Model